## СОЕДИНЕНИЕ ТАБЛИЦ

``` sql
-- Новый дата-сет про футбол (команды)

select *
from sql.teams

-- Новый дата-сет про футбол (матчи)

select *
from sql.matches

```

Таблица teams с данными о командах

* id - id команды

* api_id - ключ на таблицу matches

* long_name - полное название команды

* short_name - сокращённое название команды


Таблица matches с данными о матчах

* id - id матча

* season - сезон

* date - дата матча

* home_team_api_id - api_id домашней команды, ключ на таблицу teams

* away_team_api_id - api_id гостевой команды, ключ на таблицу teams

* home_team_goals - количество голов домашней команды

* away_team_goals - количество голов гостевой команды

``` sql

-- Сколько в таблице teams команд с коротким названием VAL? В ответе укажите только число.

select
short_name,
count(short_name)
from sql.teams
group by short_name
having short_name = 'VAL'

-- Информацию о скольких матчах содержит таблица matches? В ответе укажите только число.

select
count(*)
from sql.matches

-- Данные за какие сезоны даны в таблице matches. Ответ введите в формате 2019/2020.

select 
min(season) as min,
max(season) as max
from sql.matches

```

Существует несколько способов соединения таблиц. Начнём с простого метода объединения таблиц — без операторов.

Это действие также называют декартовым произведением таблиц.

``` sql
select *
from
sql.matches,
sql.teams
```
Однако в таком соединении особого смысла нет, потому что команды не соответствуют матчам. Поэтому добавим к нашему запросу условие по которому будут соединены таблицы:

``` sql

-- Данные о гостевых командах
select *
from
sql.matches,
sql.teams
where away_team_api_id = api_id

-- Данные о домашних командах
select *
from
sql.matches,
sql.teams
where home_team_api_id = api_id
```

В таблицах есть два вида ключа:

* primary key - первичный ключ определяет текущую таблицу и идет первым среди ключей (всегда уникален: повторяющихся значений в основной таблице быть не может)

* foreign key - внешний ключ это ссылка на другую таблицу, по этим ключам обе таблицы соединены.

Как правило, названия ключей имеют «хвост», который позволяет их идентифицировать: например, _id, _rk, _cd, _pk (от primary_key), _fk (от foreign_key) и другие.

``` sql
-- Запрос, который выведет количество строк соединённой таблицы.

select
count(*)
from 
sql.teams,
sql.matches

-- Запрос, который выведет таблицу с результатами матчей для гостевых команд, содержащую:
-- названия гостевых команд (long_name),
-- количество забитых мячей домашней команды (home_team_goals),
-- количество забитых мячей гостевой команды (away_team_goals).

select
long_name,
home_team_goals,
away_team_goals
from 
sql.teams,
sql.matches
where away_team_api_id = api_id

```

## Знакомство с join

join — это оператор SQL, который позволяет соединять таблицы по условию.

``` sql

-- ОПЕРАТОР join

select 
matches.id as id_1,
teams.id as id_2
from sql.teams
join sql.matches on home_team_api_id = api_id

-- Если необходимо записать название, в котором используются пробелы ("table 1"), то алиас можно обернуть в кавычки.

-- Важно! Обращаться по такому алиасу придётся также с помощью кавычек.

select
	"table 1".столбец1,
	"table 2".столбец2,
	...
from
	таблица1 as "table 1"
	join таблица2 as "table 2" on условие

-- Можно вместо названий столбцов искользовать их alias, например

select 
"table_1".id as id_1,
"table_2".id as id_2
from 
sql.matches as "table_1"
join sql.teams as "table_2" on home_team_api_id = api_id

Примечание: Использование таких алиасов считается плохой практикой как минимум по причине того, что обращаться с такими алиасами неудобно!

```
Давайте с помощью запроса SQL получим таблицу, содержащую:

* название домашней команды;

* количество забитых домашней командой голов;

* количество забитых гостевой командой голов;

* название гостевой команды.

``` sql
select
t.long_name as "home team name",
m.home_team_goals as "home goals",
m.away_team_goals as "away goals",
a.long_name as "away team name"
from
sql.matches as m 
join sql.teams as t on m.home_team_api_id = t.api_id
join sql.teams as a on m.away_team_api_id = a.api_id

```


## СИНТАКСИС

Оператор join упрощает процесс соединения таблиц.

Его синтаксис можно представить следующим образом:

``` sql
select
        столбец1,
	столбец2,
	...
from
	таблица1
join таблица2 on условие
```

Порядок присоединения таблиц в данном случае не важен — результат будет одинаковым.

С помощью join можно соединить и более двух таблиц.

``` sql
select
        столбец1,
	столбец2,
	...
from
	таблица1
join таблица2 on условие
join таблица3 on условие
```

В таблицах, которые мы соединяем, могут быть одинаковые названия столбцов.

``` sql

-- Запрос, который выведет два столбца: id матча (match_id) и id домашней команды (team_id). Отсортируйте по id матча в порядке возрастания значений.

select
m.id as match_id,
t.id as team_id
from
sql.matches as m
join sql.teams as t on m.home_team_api_id = t.api_id
order by m.id asc

-- Запрос, который выведет столбцы:
-- id матча,
-- короткое название домашней команды (home_short),
-- короткое название гостевой команды (away_short).
-- Отсортируйте запрос по возрастанию id матча.

select
m.id,
t.short_name as home_short,
a.short_name as away_short
from 
sql.matches as m
join sql.teams as t on m.home_team_api_id = t.api_id
join sql.teams as a on m.away_team_api_id = a.api_id
order by m.id asc
```

## РАБОТА С ОБЪЕДИНЁННЫМИ ТАБЛИЦАМИ

Принципиальное отличие фильтрации данных по соединённым таблицам от аналогичного действия по одиночным таблицам заключается в том, что, фильтруя записи одной таблицы, мы также будем фильтровать и записи другой таблицы, поскольку соединённые на уровне запроса таблицы по сути являются единой таблицей.

``` sql

-- Фильтрация данных с объединенными таблицами, например отфильтруем все матчи из таблицы матчи где в игре участвовал Arsenal

select
m.id
from
sql.teams t
join sql.matches m on m.away_team_api_id = t.api_id
where long_name = 'Arsenal'

-- Еще пример фильтрации в которых короткое название домашней команды GEN и матчи сезона 2008/2009

select 
*
from
sql.teams t
join sql.matches m on t.api_id = m.home_team_api_id
where t.short_name = 'GEN'
and m.season = '2008/2009'

-- Запрос где используются агрегатные функции. В данном случае мы суммируем все голы где команда была гостевой

Примечание: В данном запросе была использована группировка по столбцу id таблицы teams, хотя этот столбец не выводится в запросе. Это необходимо для того, чтобы команды с одинаковым названием, если такие найдутся, не группировались между собой. Группировка по названию команды в данном запросе будет неверной, так как есть несколько команд с одинаковым полным названием.

select
t.long_name,
sum(m.home_team_goals) + sum(m.away_team_goals) matches_goals
from
sql.teams t 
join sql.matches m on m.away_team_api_id = t.api_id
group by t.id

-- Запрос, который выведет полное название домашней команды (long_name), 
-- количество голов домашней команды (home_goal) и количество голов гостевой команды (away_goal) в матчах, 
-- где домашней командой были команды с коротким названием 'GEN'. 
-- Отсортируйте запрос по id матча в порядке возрастания.

select
t.long_name,
m.home_team_goals,
m.away_team_goals
from
sql.teams t 
join sql.matches m on m.home_team_api_id = t.api_id
where t.short_name = 'GEN'
order by m.id asc

-- Запрос, чтобы вывести id матчей, короткое название домашней команды (home_short), 
-- короткое название гостевой команды (away_short) для матчей сезона 2011/2012, 
-- в которых участвовала команда с названием Liverpool. 
-- Отсортируйте по id матча в порядке возрастания.

select
m.id,
h.short_name home_short,
a.short_name away_short
from
sql.matches m
join sql.teams h on m.home_team_api_id = h.api_id
join sql.teams a on m.away_team_api_id = a.api_id
where m.season = '2011/2012' and (h.long_name = 'Liverpool' or a.long_name = 'Liverpool')
order by m.id asc

```

Можем использовать оператор HAVING для фильтрации сгруппированных данных.

Поставим задачу — вывести таблицу с суммарным количеством забитых голов в матчах по командам и сезонам для команд, в которых суммарное количество голов в матчах сезона больше 100.

``` sql
select
m.season,
sum(m.home_team_goals) + sum(m.away_team_goals) total_matches
from
sql.matches m 
group by m.season
```

Затем добавим таблицу с командами, группировку по командам и условие фильтрации.

``` sql

select
m.season,
t.long_name,
sum(m.home_team_goals) + sum(m.away_team_goals) total_matches
from
sql.matches m
join sql.teams t on m.home_team_api_id = t.api_id or m.away_team_api_id = t.api_id
group by m.season, t.id
having sum(m.home_team_goals) + sum(m.away_team_goals) > 100

-- Запрос, с помощью которого можно вывести список полных названий команд, сыгравших в гостях 150 и более матчей. 
-- Отсортируйте список по названию команды.

select
t.long_name
from
sql.teams t 
join sql.matches m on m.away_team_api_id = t.api_id
group by t.id
having count(m.id) >= 150
order by t.long_name
```

## <center>ВИДЫ JOIN</center>

## INNER JOIN

INNER JOIN — это тот же JOIN (слово inner в операторе можно опустить).

Это единственный из видов JOIN который может сделать выборку короче (можно использовать в качестве фильтра, так как этот вид учитывает только то, что присутствует в обеих таблицах - пересечение множеств). Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

``` sql

-- INNER JOIN (количество команд в таблице teams)

select 
count(distinct t.api_id)
from 
sql.teams t
inner join sql.matches m on m.home_team_api_id = t.api_id or m.away_team_api_id = t.api_id

```

## LEFT JOIN

К таблица слева остается полной, а к ней присоединяются все записи, которые отвечают условию соединения. Выбираются 100 процентов всех данных базовой таблицы (слева).

Тут существуют варианты, когда таблица может увеличиться.

``` sql

-- LEFT JOIN (вывод полного названия команд, данных по которым нет в талице matches)

select
t.long_name
from
sql.teams t
left join sql.matches m on m.home_team_api_id = t.api_id or m.away_team_api_id = t.api_id
where m.id is null

-- Агрегатные функции с LEFT JOIN (вывести сумму голов команд по гостевым матчам, которые не участвовали ни в одном матче)

select
t.long_name,
sum(m.away_team_goals) total_goals
from
sql.teams t
left join sql.matches m on m.away_team_api_id = t.api_id
group by t.id
order by total_goals


-- Используя LEFT JOIN, выведите список уникальных названий команд, содержащихся в таблице matches. Отсортируйте список в алфавитном порядке.

select
distinct t.long_name
from
sql.teams t
left join sql.matches m on t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
where m.id is not null
order by t.long_name asc

-- Используя LEFT JOIN, запрос, который выведет полное название команды (long_name), 
-- количество матчей, в которых участвовала команда, — домашних и гостевых (matches_cnt). 
-- Отсортируйте по количеству матчей в порядке возрастания, затем — по названию команды в алфавитном порядке.

select
t.long_name,
count(m.home_team_goals + m.away_team_goals) matches_cnt
from
sql.teams t
left join sql.matches m on t.api_id = m.home_team_api_id or t.api_id = m.away_team_api_id
group by t.id
order by matches_cnt asc, t.long_name

```

Примечание: При использовании функций **SUM, MIN, MAX, AVERAGE** - результат **NULL**, при использовании **COUNT**, результат **0 (ноль)**.

## RIGHT JOIN

Тоже самое, что и LEFT JOIN только работает в другую сторону, когда выбираются 100 процентов всех данных базовой таблицы (справа).

Вообще, применение RIGHT JOIN считается плохим тоном, так как язык SQL читается и пишется слева направо, а такой оператор усложняет чтение запросов.

``` sql

-- Агрегатные функции с RIGHT JOIN (вывести сумму голов команд по гостевым матчам, которые не участвовали ни в одном матче)

select
t.long_name,
sum(m.away_team_goals) total_goals
from
sql.matches m
right join  sql.teams t on m.away_team_api_id = t.api_id
group by t.id
order by total_goals

```

## FULL JOIN

Это полная выборка из обеих таблиц. Скорее всего при таком соединении мы получим "дырявую" выборку, так как с обеих сторон таблиц будут пропуски.

- Слева будут пропуски, если справа не найдется записей;

- Справа тоже будут пропуски, если слева не было найдено записей.

Иными словами, где получится СУБД совместит записи и они будут нормальными, где не получится там будут пропуски: NULL, NAN и т.д. Даже если не будет соответствий, мы сохраним все записи из обеих таблиц.

``` sql

-- FULL JOIN (выведим полное название команды и id матча, в котором команда выступала дома)

select
t.long_name,
m.id
from
sql.teams t
full join sql.matches m on t.api_id = m.home_team_api_id

```

## CROSS JOIN

Это единственный вид JOIN который не использует признак соединения - ON. Как правило используется если нужно добавить фиксированную информацию или колонку к таблице.

Иными словами этот вид JOIN перемножает таблицы.

``` sql

-- Обычный запрос, указываем через запятую две таблицы

select 
*
from
sql.teams,
sql.matches

-- Та же запись но с CROSS JOIN

select *
from
sql.teams
cross join sql.matches

-- CROSS JOIN (выведим все возможные комбинации команд)

select
distinct -- чтобы убрать повторения
t_1.long_name fisrt_team,
t_2.long_name second_team
from
sql.teams t_1
cross join sql.teams t_2 -- пересекаем таблицу саму на себя

-- Запрос, который выведет все возможные уникальные комбинации коротких названий домашней команды (home_team) и 
-- коротких названий гостевой команды (away_team). Команда не может сама с собой играть, то есть быть и домашней, и 
-- одновременно гостевой (в одном и том же матче).
-- Отсортируйте запрос по первому и второму столбцам.

select
distinct
h_t.short_name home_team,
a_t.short_name away_team
from
sql.teams h_t
cross join sql.teams a_t
where h_t.id != a_t.id -- условие именно по ID
order by 1, 2

```

## NATURAL JOIN

Работает так же как и INNER JOIN, писать условие ON не нужно, соединение будет происходить по колонкам которые называются одинаково.

## ПРАКТИКА

``` sql
-- Запрос, который выведет список уникальных полных названий команд (long_name), игравших в гостях в матчах сезона 2012/2013. Отсортируйте список в алфавитном порядке.

select
distinct
t.long_name
from
sql.teams t
inner join sql.matches m on t.api_id = m.away_team_api_id
where m.season = '2012/2013'
order by t.long_name

-- Запрос, который выведет полное название команды (long_name) и общее количество матчей (matches_cnt), сыгранных командой Inter в домашних матчах.

select
t.long_name,
count(m.home_team_goals + m.away_team_goals) matches_cnt
from
sql.teams t
inner join sql.matches m on t.api_id = m.home_team_api_id
where t.long_name = 'Inter'
group by t.id

-- Запрос, который выведет топ-10 команд (long_name) по суммарному количеству забитых голов в гостевых матчах. Во втором столбце запроса выведите суммарное количество голов в гостевых матчах (total_goals).

select
t.long_name,
sum(m.away_team_goals) total_goals
from
sql.teams t
inner join sql.matches m on t.api_id = m.away_team_api_id
group by t.id
order by total_goals desc
limit 10

-- Выведите количество матчей между командами Real Madrid CF и FC Barcelona. В поле ниже введите запрос, с помощью которого вы решили задание.

select
count(*)
from
sql.matches m
join sql.teams t_1 on m.home_team_api_id = t_1.api_id
join sql.teams t_2 on m.away_team_api_id = t_2.api_id
where (t_1.long_name = 'Real Madrid CF' and t_2.long_name = 'FC Barcelona') 
or (t_1.long_name = 'FC Barcelona' and t_2.long_name = 'Real Madrid CF')

-- Запрос, который выведет название команды (long_name), сезон (season) и суммарное количество забитых голов в домашних матчах (total_goals). 
-- Оставьте только те строки, в которых суммарное количество голов менее десяти. Отсортируйте запрос по названию команды, а затем — по сезону.

select
t.long_name,
m.season,
sum(m.home_team_goals) total_goals
from
sql.teams t
inner join sql.matches m on t.api_id = m.home_team_api_id
group by t.long_name, m.season
having sum(m.home_team_goals) < 10
order by t.long_name, m.season
```